# Podešavanje hiperparametara

U ovom primeru klasifikacije, upoznaćemo proceduru pronalaženja optimalnih vrednosti hiperparametara kernelizovane SVM metode. Isti princip se primenjuje i u određivanju optimalnih vrednosti hiperparametara drugih metoda klasifikacije ili regresije. Pošto je ovaj zadatak vrlo važan, bavićemo se njime i još nekim sličnim procedurama u priči o regularizacijama i evaluaciji.  

In [1]:
import numpy as np
import pandas as pd

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
from sklearn import svm
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets
from sklearn import preprocessing

U radu ćemo koristiti Viskonsis skup podataka za klasifikaciju tumora na benigne i maligne koji smo koristili i u primeru sa logističkom regresijom.

In [4]:
data = datasets.load_breast_cancer()

In [5]:
#print(data.DESCR)

Podsetimo se da skup podataka ima 30 atributa numeričkog tipa i ciljnu promenljivu sa vrednostima 0 i 1. 

In [6]:
len(data.feature_names)

30

In [7]:
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [8]:
X = data.data

In [9]:
X.shape

(569, 30)

In [10]:
y = data.target

In [11]:
np.unique(y)

array([0, 1])

Kada želimo da odredimo optimalne vrednosti hiperparametara, iz skupa za treniranje izdvajamo jedan manji skup podataka koji zovemo validacionim skupom. Nad njime dalje proveravamo kako se ponaša model koji razvijamo za različite vrednosti hiperparameta i biramo one hiperparametre koji daju najbolje rezultate u nekom smislu. Pogrešili bismo da za ovaj zadatak iskoristimo skup sa testiranje jer bi ocena koju bi dobili kasnije prilikom testiranja modela sa odabranim vrednostima hiperparametara bila pristrasna.

Delimo podatke na skup za treniranje, validaciju i testiranje. 

<img src='assets/train_validation_test.png'>

In [12]:
X_train_and_validation, X_test, y_train_and_validation, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y )

In [13]:
X_train, X_validation, y_train, y_validation = model_selection.train_test_split(X_train_and_validation, y_train_and_validation, train_size = 0.8, random_state = 42, stratify = y_train_and_validation )

Standardizujemo podatke. 

In [14]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

Hiperparametri kernelizovanog SVM modela koje ćemo podešavati su `C` i `gamma`. Oni, redom, kontrolišu jačinu regularizacije i širinu (engl. bandwidth) RBF kernela. 

In [15]:
# jačina regularizacije je inverzno proporcionalna parametru C tj. uzima vrednost 1/C
# C mora biti strogo pozitivan broj 
# podrazumevano se koristi kvadrat l2 regularizacije
Cs = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

In [16]:
gammas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

Inicijalizujemo promenljive u kojima ćemo čuvati najbolje ocene i optimalne vrednosti hiperparametara.

In [17]:
best_f1_score = 0
best_C = None
best_gamma = None

Za svaki par vrednosti `C` i `gamma` treniramo model na trening skupu i pratimo njegovu ocenu na validacionom skupu.

In [18]:
for C in Cs:
    for gamma in gammas:
        model = svm.SVC(kernel='rbf', gamma=gamma, C = C)
        model.fit(X_train, y_train)
        f1_score = metrics.f1_score(y_validation, model.predict(X_validation))
        if f1_score > best_f1_score:
            best_f1_score = f1_score
            best_C = C
            best_gamma = gamma

In [19]:
print('Najbolji f1-score na validacionom skupu je: ', best_f1_score)

Najbolji f1-score na validacionom skupu je:  0.9900990099009901


In [20]:
print('Najbolji hiperparametri modela su: ', best_C, best_gamma)

Najbolji hiperparametri modela su:  1 0.1


Kreiramo konačni model: 

In [21]:
best_model = svm.SVC(kernel='rbf', gamma=best_gamma, C=best_C)
best_model.fit(X_train, y_train)

SVC(C=1, gamma=0.1)

I na kraju ispravno testiramo model: 

In [22]:
f1_score = metrics.f1_score(y_test, best_model.predict(X_test))

In [23]:
print('f1-score na test skupu je: ', f1_score)

f1-score na test skupu je:  0.966824644549763


In [24]:
print(metrics.classification_report(y_test, best_model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95        64
           1       0.98      0.95      0.97       107

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



U delu o evaluaciji upoznaćemo postupak unakrsne validacije i `GridSearchCV` metod koji će zameniti ovaj repetitivni posao i odgovoriti na pitanje "Da li bi se ocena modela promenila za nešto drugačije izbore validacionih i test skupova?". 